In [3]:
import pandas as pd
import numpy as np

In [4]:
test_data = pd.read_csv('./data/public_test.csv')
train_data = pd.read_csv('./data/public_train.csv')

C:\Programs\miniconda\envs\dap2020_jupyter\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (44,45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Segédfüggvények

In [5]:
def limitalt_dummy_valtozo(indf, oszlop, limit, drop_last=True):
    uj_valtozok = []
    df = indf.copy()
    egyedi_elemszam=df[oszlop].nunique()
    if egyedi_elemszam <= limit and drop_last is True:
        egyedi_elemszam = egyedi_elemszam - 1
    tmp = df[oszlop].value_counts()[:limit]
    for value in list(tmp.index):
        def f(x):
            if x == value:
                return 1
            else:
                return 0
        df[oszlop+'='+str(value)] = df[oszlop].apply(f)
        uj_valtozok.append(oszlop + '=' + str(value))
    return df, uj_valtozok

In [6]:
from sklearn.metrics import roc_auc_score

def myxval(models,train_df,bemeno_valtozok,fold_num=3):
    train_df['xval']=train_df['ID']%fold_num
    mean_auc_list=[]
    for i in range(fold_num):
        mini_train=train_df[train_df['xval']!=i].copy()
        mini_test=train_df[train_df['xval']==i].copy()
        for model in models:
            auc_list = []
            model.fit(mini_train[bemeno_valtozok],mini_train['TARGET'])
            mini_test['p1']=model.predict_proba(mini_test[bemeno_valtozok])[:,1]
            auc=roc_auc_score(mini_test['TARGET'],mini_test['p1'])
            auc_list.append(auc)
        mean_auc = np.mean(auc_list)
        mean_auc_list.append(mean_auc)
    return np.mean(mean_auc_list)

# Városok adatainak hozzáfűzése

In [7]:
test_data['train_test'] = 'test'
train_data['train_test'] = 'train'

df = pd.concat([train_data, test_data])

#### RES_CITY

In [8]:
nonan = train_data.copy()
nonan['RES_CITY'] = nonan['RES_CITY'].fillna('nincs_adat')

In [9]:
seged = nonan.groupby(['RES_CITY'], as_index=False).agg({'TARGET':'mean', 'ID':'count'})
seged.columns = ['RES_CITY', 'MEAN_RES_CITY', 'COUNT']

In [10]:
mean = seged['MEAN_RES_CITY'].mean()
seged['MEAN_RES_CITY'] = seged['MEAN_RES_CITY'].fillna(mean)
for index, row in seged.iterrows():
    if row['COUNT'] < 6:
        seged.loc[index, 'MEAN_RES_CITY'] = mean

In [11]:
df_merged = df.merge(seged[['RES_CITY', 'MEAN_RES_CITY']], on='RES_CITY', how='left')
df_merged['MEAN_RES_CITY'] = df_merged['MEAN_RES_CITY'].fillna(mean)

In [12]:
df_merged
df_merged.describe()

,ID,DAY_OF_PAYMENT,M_STATUS,NUMBER_OF_DEPENDANTS,NAT,RES_TYPE,MONTHS_IN_RES,EMAIL_FLAG,P_MONTHLY_INCOME,O_INCOMES,...,BILL_AMOUNT_MONTH_5,BILL_AMOUNT_MONTH_6,PAY_AMOUNT_MONTH_1,PAY_AMOUNT_MONTH_2,PAY_AMOUNT_MONTH_3,PAY_AMOUNT_MONTH_4,PAY_AMOUNT_MONTH_5,PAY_AMOUNT_MONTH_6,TARGET,MEAN_RES_CITY
count,50000.000000,50000.000000,50000.00000,50000.000000,50000.000000,48651.000000,46223.000000,50000.000000,49472.000000,50000.000000,...,50000.000000,50000.000000,50000.0000,5.000000e+04,50000.000000,50000.00000,50000.000000,50000.000000,25000.000000,50000.000000
mean,25000.500000,12.869920,2.14840,0.650520,0.961600,1.252225,9.727149,0.802280,180633.123690,2963.729060,...,45637.257120,44628.478760,5619.1078,5.436837e+03,5146.842240,5568.31044,5326.496520,4930.801480,0.424320,0.421255
std,14433.901067,6.608385,1.32285,1.193655,0.202105,0.867833,10.668841,0.398284,134589.611245,15782.379761,...,61091.066392,60077.818698,16557.9558,1.724584e+04,18372.905193,22290.83708,18960.915332,16648.887358,0.494249,0.088352
min,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,10000.000000,0.000000,...,-37594.000000,-51443.000000,0.0000,0.000000e+00,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,12500.750000,10.000000,1.00000,0.000000,1.000000,1.000000,1.000000,1.000000,94000.000000,0.000000,...,2475.000000,2160.750000,1200.0000,1.023000e+03,700.000000,500.00000,500.000000,416.000000,0.000000,0.389671
50%,25000.500000,10.000000,2.00000,0.000000,1.000000,1.000000,6.000000,1.000000,133200.000000,0.000000,...,26152.000000,25833.000000,2784.0000,2.438000e+03,2000.000000,1959.00000,2000.000000,1894.000000,0.000000,0.405204
75%,37500.250000,15.000000,2.00000,1.000000,1.000000,1.000000,15.000000,1.000000,215576.000000,0.000000,...,66895.000000,65239.000000,5000.0000,5.000000e+03,4407.000000,4059.00000,4000.000000,3900.000000,1.000000,0.455285
max,50000.000000,25.000000,7.00000,53.000000,2.000000,5.000000,228.000000,1.000000,999976.400000,200000.000000,...,927171.000000,961664.000000,873552.0000,1.215471e+06,896040.000000,621000.00000,417990.000000,528666.000000,1.000000,0.916667


#### PROF_CITY

In [13]:
nonan = train_data.copy()
nonan['PROF_CITY'] = nonan['PROF_CITY'].fillna('nincs_adat')

In [14]:
seged = nonan.groupby(['PROF_CITY'], as_index=False).agg({'TARGET':'mean', 'ID':'count'})
seged.columns = ['PROF_CITY', 'MEAN_PROF_CITY', 'COUNT']

In [15]:
mean = seged['MEAN_PROF_CITY'].mean()
seged['MEAN_PROF_CITY'] = seged['MEAN_PROF_CITY'].fillna(mean)
for index, row in seged.iterrows():
    if row['COUNT'] < 6:
        seged.loc[index, 'MEAN_PROF_CITY'] = mean

In [16]:
df_merged2 = df_merged.merge(seged[['PROF_CITY', 'MEAN_PROF_CITY']], on='PROF_CITY', how='left')
df_merged2['MEAN_PROF_CITY'] = df_merged2['MEAN_PROF_CITY'].fillna(mean)

In [17]:
df_merged2

,ID,DAY_OF_PAYMENT,APP_SUB_TYPE,P_ADDRESS_TYPE,SEX,M_STATUS,NUMBER_OF_DEPENDANTS,BIRTH_STATE,BIRTH_CITY,NAT,...,PAY_AMOUNT_MONTH_1,PAY_AMOUNT_MONTH_2,PAY_AMOUNT_MONTH_3,PAY_AMOUNT_MONTH_4,PAY_AMOUNT_MONTH_5,PAY_AMOUNT_MONTH_6,TARGET,train_test,MEAN_RES_CITY,MEAN_PROF_CITY
0,1,5,W,A,female,6,1,State0,City0,1,...,26034,3182,49174,11823,669,1534,1.0,train,0.405204,0.398107
1,2,15,C,A,female,2,0,State1,City1,1,...,9861,11021,16883,150,4661,873,1.0,train,0.371105,0.398107
2,3,20,W,A,female,2,0,State2,City3,1,...,5100,5500,5170,3543,3000,3238,0.0,train,0.520000,0.398107
3,4,25,W,A,female,2,0,State7,City9,1,...,3855,0,1694,3713,2000,1900,1.0,train,0.455285,0.398107
4,5,15,W,A,female,1,1,State3,City10,1,...,4780,0,0,0,0,0,0.0,train,0.350877,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,10,W,A,female,2,2,State9,NaN,1,...,7000,4710,4500,9400,3600,5500,NaN,test,0.555556,0.398107
49996,49997,15,W,A,female,1,0,State7,NaN,1,...,0,3126,2578,5616,0,3000,NaN,test,0.368421,0.398107
49997,49998,15,Z,A,female,1,1,State0,City32,1,...,0,0,0,0,0,0,NaN,test,0.405204,0.398107
49998,49999,15,W,A,male,2,0,State5,City1442,1,...,12,1034,727,0,13,1330,NaN,test,0.423077,0.398107


# TO OBJECT

In [18]:
df_merged2.columns

Index(['ID', 'DAY_OF_PAYMENT', 'APP_SUB_TYPE', 'P_ADDRESS_TYPE', 'SEX',
       'M_STATUS', 'NUMBER_OF_DEPENDANTS', 'BIRTH_STATE', 'BIRTH_CITY', 'NAT',
       'RES_STATE', 'RES_CITY', 'RES_BOROUGH', 'HOME_PHONE_FLAG',
       'RES_PHONE_AREA_CODE', 'RES_TYPE', 'MONTHS_IN_RES', 'EMAIL_FLAG',
       'P_MONTHLY_INCOME', 'O_INCOMES', 'VISA_FLAG', 'MCARD_FLAG', 'S_FLAG',
       'AME_FLAG', 'OCARDS_FLAG', 'NUMBERO_OF_BANK_ACCOUNTS',
       'NUMBER_OF_SPECIAL_BANK_ACCOUNTS', 'P_ASSETS_VALUE', 'CARS_FLAG',
       'C_NAME_GIVEN', 'PROF_STATE', 'PROF_CITY', 'PROF_BOROUGH',
       'PROF_PHONE_GIVEN', 'PROF_PHONE_AREA_CODE', 'M_IN_THE_JOB', 'PROF_CODE',
       'OCC_TYPE', 'MATE_PROF_CODE', 'EDU_LEVEL', 'CONF_CITIZEN_CARD_NUM',
       'CONF_TAX_STATUS', 'C_PRODUCT', 'AGE', 'RES_ZIP', 'PROF_ZIP',
       'L_BALANCE', 'PAY_MONTH_0', 'PAY_MONTH_2', 'PAY_MONTH_3', 'PAY_MONTH_4',
       'PAY_MONTH_5', 'PAY_MONTH_6', 'BILL_AMOUNT_MONTH_1',
       'BILL_AMOUNT_MONTH_2', 'BILL_AMOUNT_MONTH_3', 'BILL_AMOUNT_MO

In [19]:
object_valtozok = ['NAT', 'M_STATUS', 'MATE_PROF_CODE', 'VISA_FLAG', 'MCARD_FLAG', 'S_FLAG',
       'AME_FLAG', 'OCARDS_FLAG', 'CARS_FLAG', 'PROF_CODE', 'OCC_TYPE', 'C_PRODUCT', 'RES_TYPE']

In [20]:
for att in object_valtozok:
    df_merged2[att] = df_merged2[att].astype('O')

In [21]:
df_merged2

,ID,DAY_OF_PAYMENT,APP_SUB_TYPE,P_ADDRESS_TYPE,SEX,M_STATUS,NUMBER_OF_DEPENDANTS,BIRTH_STATE,BIRTH_CITY,NAT,...,PAY_AMOUNT_MONTH_1,PAY_AMOUNT_MONTH_2,PAY_AMOUNT_MONTH_3,PAY_AMOUNT_MONTH_4,PAY_AMOUNT_MONTH_5,PAY_AMOUNT_MONTH_6,TARGET,train_test,MEAN_RES_CITY,MEAN_PROF_CITY
0,1,5,W,A,female,6,1,State0,City0,1,...,26034,3182,49174,11823,669,1534,1.0,train,0.405204,0.398107
1,2,15,C,A,female,2,0,State1,City1,1,...,9861,11021,16883,150,4661,873,1.0,train,0.371105,0.398107
2,3,20,W,A,female,2,0,State2,City3,1,...,5100,5500,5170,3543,3000,3238,0.0,train,0.520000,0.398107
3,4,25,W,A,female,2,0,State7,City9,1,...,3855,0,1694,3713,2000,1900,1.0,train,0.455285,0.398107
4,5,15,W,A,female,1,1,State3,City10,1,...,4780,0,0,0,0,0,0.0,train,0.350877,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,10,W,A,female,2,2,State9,NaN,1,...,7000,4710,4500,9400,3600,5500,NaN,test,0.555556,0.398107
49996,49997,15,W,A,female,1,0,State7,NaN,1,...,0,3126,2578,5616,0,3000,NaN,test,0.368421,0.398107
49997,49998,15,Z,A,female,1,1,State0,City32,1,...,0,0,0,0,0,0,NaN,test,0.405204,0.398107
49998,49999,15,W,A,male,2,0,State5,City1442,1,...,12,1034,727,0,13,1330,NaN,test,0.423077,0.398107


# Fizetési adatok

In [22]:
df_merged2['MEAN_BILL_AMOUNT_MONTH'] = (df_merged2['BILL_AMOUNT_MONTH_1'] + df_merged2['BILL_AMOUNT_MONTH_2'] 
                                + df_merged2['BILL_AMOUNT_MONTH_3'] + df_merged2['BILL_AMOUNT_MONTH_4']
                                + df_merged2['BILL_AMOUNT_MONTH_5'] + df_merged2['BILL_AMOUNT_MONTH_6']) / 6

In [23]:
df_merged2['MEAN_PAY_MONTH'] = (df_merged2['PAY_MONTH_0'] + df_merged2['PAY_MONTH_2'] + df_merged2['PAY_MONTH_3'] 
                        + df_merged2['PAY_MONTH_4'] + df_merged2['PAY_MONTH_5'] + df_merged2['PAY_MONTH_6']) / 6

In [24]:
df_merged2['MEAN_PAY_AMOUNT_MONTH'] = (df_merged2['PAY_AMOUNT_MONTH_1'] + df_merged2['PAY_AMOUNT_MONTH_2'] 
                               + df_merged2['PAY_AMOUNT_MONTH_3'] + df_merged2['PAY_AMOUNT_MONTH_4']
                               + df_merged2['PAY_AMOUNT_MONTH_5'] + df_merged2['PAY_AMOUNT_MONTH_6']) / 6

In [25]:
df_merged2

,ID,DAY_OF_PAYMENT,APP_SUB_TYPE,P_ADDRESS_TYPE,SEX,M_STATUS,NUMBER_OF_DEPENDANTS,BIRTH_STATE,BIRTH_CITY,NAT,...,PAY_AMOUNT_MONTH_4,PAY_AMOUNT_MONTH_5,PAY_AMOUNT_MONTH_6,TARGET,train_test,MEAN_RES_CITY,MEAN_PROF_CITY,MEAN_BILL_AMOUNT_MONTH,MEAN_PAY_MONTH,MEAN_PAY_AMOUNT_MONTH
0,1,5,W,A,female,6,1,State0,City0,1,...,11823,669,1534,1.0,train,0.405204,0.398107,15835.333333,-2.000000,15402.666667
1,2,15,C,A,female,2,0,State1,City1,1,...,150,4661,873,1.0,train,0.371105,0.398107,7946.000000,-2.000000,7241.500000
2,3,20,W,A,female,2,0,State2,City3,1,...,3543,3000,3238,0.0,train,0.520000,0.398107,93721.500000,0.000000,4258.500000
3,4,25,W,A,female,2,0,State7,City9,1,...,3713,2000,1900,1.0,train,0.455285,0.398107,48116.833333,0.666667,2193.666667
4,5,15,W,A,female,1,1,State3,City10,1,...,0,0,0,0.0,train,0.350877,0.333333,361.666667,-2.000000,796.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,10,W,A,female,2,2,State9,NaN,1,...,9400,3600,5500,NaN,test,0.555556,0.398107,128566.500000,0.333333,5785.000000
49996,49997,15,W,A,female,1,0,State7,NaN,1,...,5616,0,3000,NaN,test,0.368421,0.398107,71559.500000,1.000000,2386.666667
49997,49998,15,Z,A,female,1,1,State0,City32,1,...,0,0,0,NaN,test,0.405204,0.398107,998.000000,-1.333333,0.000000
49998,49999,15,W,A,male,2,0,State5,City1442,1,...,0,13,1330,NaN,test,0.423077,0.398107,2196.333333,-0.166667,519.333333


# Változók szűrése

In [26]:
df.columns

Index(['ID', 'DAY_OF_PAYMENT', 'APP_SUB_TYPE', 'P_ADDRESS_TYPE', 'SEX',
       'M_STATUS', 'NUMBER_OF_DEPENDANTS', 'BIRTH_STATE', 'BIRTH_CITY', 'NAT',
       'RES_STATE', 'RES_CITY', 'RES_BOROUGH', 'HOME_PHONE_FLAG',
       'RES_PHONE_AREA_CODE', 'RES_TYPE', 'MONTHS_IN_RES', 'EMAIL_FLAG',
       'P_MONTHLY_INCOME', 'O_INCOMES', 'VISA_FLAG', 'MCARD_FLAG', 'S_FLAG',
       'AME_FLAG', 'OCARDS_FLAG', 'NUMBERO_OF_BANK_ACCOUNTS',
       'NUMBER_OF_SPECIAL_BANK_ACCOUNTS', 'P_ASSETS_VALUE', 'CARS_FLAG',
       'C_NAME_GIVEN', 'PROF_STATE', 'PROF_CITY', 'PROF_BOROUGH',
       'PROF_PHONE_GIVEN', 'PROF_PHONE_AREA_CODE', 'M_IN_THE_JOB', 'PROF_CODE',
       'OCC_TYPE', 'MATE_PROF_CODE', 'EDU_LEVEL', 'CONF_CITIZEN_CARD_NUM',
       'CONF_TAX_STATUS', 'C_PRODUCT', 'AGE', 'RES_ZIP', 'PROF_ZIP',
       'L_BALANCE', 'PAY_MONTH_0', 'PAY_MONTH_2', 'PAY_MONTH_3', 'PAY_MONTH_4',
       'PAY_MONTH_5', 'PAY_MONTH_6', 'BILL_AMOUNT_MONTH_1',
       'BILL_AMOUNT_MONTH_2', 'BILL_AMOUNT_MONTH_3', 'BILL_AMOUNT_MO

In [27]:
bemeno_valtozok = ['DAY_OF_PAYMENT', 'APP_SUB_TYPE', 'P_ADDRESS_TYPE', 'SEX', 
                   'M_STATUS', 'NUMBER_OF_DEPENDANTS', 'NAT', 'RES_TYPE', 'MONTHS_IN_RES',
                  'P_MONTHLY_INCOME', 'O_INCOMES', 'VISA_FLAG', 'MCARD_FLAG', 'S_FLAG',
       'AME_FLAG', 'OCARDS_FLAG', 'NUMBERO_OF_BANK_ACCOUNTS',
       'NUMBER_OF_SPECIAL_BANK_ACCOUNTS', 'P_ASSETS_VALUE', 'CARS_FLAG', 'C_NAME_GIVEN', 'M_IN_THE_JOB', 'PROF_CODE',
                   'OCC_TYPE', 'MATE_PROF_CODE', 
                   'C_PRODUCT', 'AGE', 'L_BALANCE', 'MEAN_PAY_AMOUNT_MONTH', 'MEAN_BILL_AMOUNT_MONTH', 'MEAN_PAY_MONTH',
                  'MEAN_RES_CITY', 'MEAN_PROF_CITY', 'EDU_LEVEL']

In [28]:
pd.set_option('display.max_columns', 100)

In [29]:
df_merged2[bemeno_valtozok].describe()

,DAY_OF_PAYMENT,NUMBER_OF_DEPENDANTS,MONTHS_IN_RES,P_MONTHLY_INCOME,O_INCOMES,NUMBERO_OF_BANK_ACCOUNTS,NUMBER_OF_SPECIAL_BANK_ACCOUNTS,P_ASSETS_VALUE,M_IN_THE_JOB,AGE,L_BALANCE,MEAN_PAY_AMOUNT_MONTH,MEAN_BILL_AMOUNT_MONTH,MEAN_PAY_MONTH,MEAN_RES_CITY,MEAN_PROF_CITY,EDU_LEVEL
count,50000.000000,50000.000000,46223.000000,49472.000000,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,17662.000000
mean,12.869920,0.650520,9.727149,180633.123690,2963.729060,0.357840,0.357840,4.451491e+05,0.009320,43.24852,164094.600000,5338.065830,51737.170600,-0.126310,0.421255,0.407483,0.296003
std,6.608385,1.193655,10.668841,134589.611245,15782.379761,0.479953,0.479953,2.281880e+06,0.383453,14.98905,123839.020397,9997.410420,61598.721738,0.915897,0.088352,0.065432,0.955688
min,1.000000,0.000000,0.000000,10000.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,6.00000,10000.000000,0.000000,-10470.833333,-2.000000,0.000000,0.000000,0.000000
25%,10.000000,0.000000,1.000000,94000.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,31.00000,80000.000000,1370.541667,7177.166667,-0.666667,0.389671,0.398107,0.000000
50%,10.000000,0.000000,6.000000,133200.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,41.00000,120000.000000,2609.833333,35977.833333,0.000000,0.405204,0.398107,0.000000
75%,15.000000,1.000000,15.000000,215576.000000,0.000000,1.000000,1.000000,0.000000e+00,0.000000,53.00000,200000.000000,5400.000000,73460.291667,0.000000,0.455285,0.398107,0.000000
max,25.000000,53.000000,228.000000,999976.400000,200000.000000,2.000000,2.000000,1.683611e+07,35.000000,106.00000,1000000.000000,627344.333333,877313.833333,5.500000,0.916667,1.000000,5.000000


# Hiányzó változók feltöltése

In [30]:
for att in bemeno_valtozok:
    if df_merged2[att].dtype=="object": 
        #print(att)
        df_merged2[att]=df_merged2[att].fillna("nincs_adat")

In [31]:
df_merged2['MONTHS_IN_RES'] = df_merged['MONTHS_IN_RES'].fillna(0)

In [32]:
df_merged2['P_MONTHLY_INCOME'] = df_merged2['P_MONTHLY_INCOME'].fillna(0)

In [33]:
df_merged2['EDU_LEVEL'] = df_merged2['EDU_LEVEL'].fillna(0)

In [34]:
df_merged2[bemeno_valtozok].describe(include='all')

,DAY_OF_PAYMENT,APP_SUB_TYPE,P_ADDRESS_TYPE,SEX,M_STATUS,NUMBER_OF_DEPENDANTS,NAT,RES_TYPE,MONTHS_IN_RES,P_MONTHLY_INCOME,O_INCOMES,VISA_FLAG,MCARD_FLAG,S_FLAG,AME_FLAG,OCARDS_FLAG,NUMBERO_OF_BANK_ACCOUNTS,NUMBER_OF_SPECIAL_BANK_ACCOUNTS,P_ASSETS_VALUE,CARS_FLAG,C_NAME_GIVEN,M_IN_THE_JOB,PROF_CODE,OCC_TYPE,MATE_PROF_CODE,C_PRODUCT,AGE,L_BALANCE,MEAN_PAY_AMOUNT_MONTH,MEAN_BILL_AMOUNT_MONTH,MEAN_PAY_MONTH,MEAN_RES_CITY,MEAN_PROF_CITY,EDU_LEVEL
count,50000.000000,50000,50000,50000,50000.00000,50000.000000,50000.000000,50000.0,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,5.000000e+04,50000.000000,50000,50000.000000,50000.0,50000.0,50000,50000.000000,50000.00000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.00000
unique,NaN,3,2,3,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,20.0,7.0,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,W,A,female,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,9.0,2.0,nincs_adat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,28206,49673,30805,NaN,NaN,NaN,41572.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27959,NaN,30092.0,16947.0,28884,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,12.869920,NaN,NaN,NaN,2.14840,0.650520,0.961600,NaN,8.992360,178725.637904,2963.729060,0.111440,0.097460,0.001320,0.001740,0.002040,0.357840,0.357840,4.451491e+05,0.336140,NaN,0.009320,NaN,NaN,NaN,1.275700,43.24852,164094.600000,5338.065830,51737.170600,-0.126310,0.421255,0.407483,0.10456
std,6.608385,NaN,NaN,NaN,1.32285,1.193655,0.202105,NaN,10.575128,135144.350172,15782.379761,0.314679,0.296586,0.036308,0.041677,0.045121,0.479953,0.479953,2.281880e+06,0.472392,NaN,0.383453,NaN,NaN,NaN,0.988286,14.98905,123839.020397,9997.410420,61598.721738,0.915897,0.088352,0.065432,0.58535
min,1.000000,NaN,NaN,NaN,0.00000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,NaN,0.000000,NaN,NaN,NaN,1.000000,6.00000,10000.000000,0.000000,-10470.833333,-2.000000,0.000000,0.000000,0.00000
25%,10.000000,NaN,NaN,NaN,1.00000,0.000000,1.000000,NaN,1.000000,91480.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,NaN,0.000000,NaN,NaN,NaN,1.000000,31.00000,80000.000000,1370.541667,7177.166667,-0.666667,0.389671,0.398107,0.00000
50%,10.000000,NaN,NaN,NaN,2.00000,0.000000,1.000000,NaN,5.000000,133200.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,NaN,0.000000,NaN,NaN,NaN,1.000000,41.00000,120000.000000,2609.833333,35977.833333,0.000000,0.405204,0.398107,0.00000
75%,15.000000,NaN,NaN,NaN,2.00000,1.000000,1.000000,NaN,14.000000,213000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000e+00,1.000000,NaN,0.000000,NaN,NaN,NaN,1.000000,53.00000,200000.000000,5400.000000,73460.291667,0.000000,0.455285,0.398107,0.00000


# Dummy változók készítése

In [35]:
object_valtozok = []
uj_valtozok = []
for att in bemeno_valtozok:
    if df_merged2[att].dtype == 'object':
        df_merged2, val = limitalt_dummy_valtozo(df_merged2, att, 5)
        object_valtozok.append(att)
        uj_valtozok.extend(val)

In [36]:
bemeno_valtozok = bemeno_valtozok + uj_valtozok
bemeno_valtozok = [val for val in bemeno_valtozok if val not in object_valtozok]

In [37]:
df_merged2[bemeno_valtozok]

,DAY_OF_PAYMENT,M_STATUS,NUMBER_OF_DEPENDANTS,NAT,MONTHS_IN_RES,P_MONTHLY_INCOME,O_INCOMES,VISA_FLAG,MCARD_FLAG,S_FLAG,AME_FLAG,OCARDS_FLAG,NUMBERO_OF_BANK_ACCOUNTS,NUMBER_OF_SPECIAL_BANK_ACCOUNTS,P_ASSETS_VALUE,CARS_FLAG,M_IN_THE_JOB,C_PRODUCT,AGE,L_BALANCE,MEAN_PAY_AMOUNT_MONTH,MEAN_BILL_AMOUNT_MONTH,MEAN_PAY_MONTH,MEAN_RES_CITY,MEAN_PROF_CITY,EDU_LEVEL,APP_SUB_TYPE=W,APP_SUB_TYPE=Z,APP_SUB_TYPE=C,P_ADDRESS_TYPE=A,P_ADDRESS_TYPE=B,SEX=female,SEX=male,SEX=nincs_adat,RES_TYPE=1.0,RES_TYPE=2.0,RES_TYPE=5.0,RES_TYPE=nincs_adat,RES_TYPE=0.0,C_NAME_GIVEN=no,C_NAME_GIVEN=yes,PROF_CODE=9.0,PROF_CODE=nincs_adat,PROF_CODE=11.0,PROF_CODE=0.0,PROF_CODE=2.0,OCC_TYPE=2.0,OCC_TYPE=1.0,OCC_TYPE=nincs_adat,OCC_TYPE=4.0,OCC_TYPE=5.0,MATE_PROF_CODE=nincs_adat,MATE_PROF_CODE=0.0,MATE_PROF_CODE=11.0,MATE_PROF_CODE=9.0,MATE_PROF_CODE=16.0
0,5,6,1,1,15.0,245200.0,0.0,1,1,0,0,0,0,0,0.0,0,0,1,32,150000,15402.666667,15835.333333,-2.000000,0.405204,0.398107,0.0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
1,15,2,0,1,1.0,203200.0,0.0,0,0,0,0,0,0,0,0.0,0,0,1,34,180000,7241.500000,7946.000000,-2.000000,0.371105,0.398107,0.0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0
2,20,2,0,1,0.0,133200.0,0.0,0,0,0,0,0,0,0,0.0,0,0,1,61,120000,4258.500000,93721.500000,0.000000,0.520000,0.398107,0.0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0
3,25,2,0,1,10.0,49200.0,0.0,0,1,0,0,0,0,0,0.0,0,0,1,36,50000,2193.666667,48116.833333,0.666667,0.455285,0.398107,0.0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0
4,15,1,1,1,6.0,106040.0,0.0,0,0,0,0,0,0,0,0.0,0,0,2,29,100000,796.666667,361.666667,-2.000000,0.350877,0.333333,0.0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,10,2,2,1,15.0,137960.0,0.0,0,1,0,0,0,0,0,0.0,0,0,1,59,130000,5785.000000,128566.500000,0.333333,0.555556,0.398107,0.0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0
49996,15,1,0,1,1.0,91200.0,0.0,0,0,0,0,0,0,0,0.0,0,0,2,38,80000,2386.666667,71559.500000,1.000000,0.368421,0.398107,0.0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0
49997,15,1,1,1,3.0,91200.0,36700.0,0,0,0,0,0,1,1,0.0,1,0,1,37,80000,0.000000,998.000000,-1.333333,0.405204,0.398107,0.0,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0
49998,15,2,0,1,5.0,142440.0,0.0,0,0,0,0,0,0,0,0.0,0,0,1,68,130000,519.333333,2196.333333,-0.166667,0.423077,0.398107,0.0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0


In [38]:
d = dict()
for att in bemeno_valtozok:
    d[att] = df_merged2[att].dtype
d

{'DAY_OF_PAYMENT': dtype('int64'),
 'M_STATUS': dtype('int64'),
 'NUMBER_OF_DEPENDANTS': dtype('int64'),
 'NAT': dtype('int64'),
 'MONTHS_IN_RES': dtype('float64'),
 'P_MONTHLY_INCOME': dtype('float64'),
 'O_INCOMES': dtype('float64'),
 'VISA_FLAG': dtype('int64'),
 'MCARD_FLAG': dtype('int64'),
 'S_FLAG': dtype('int64'),
 'AME_FLAG': dtype('int64'),
 'OCARDS_FLAG': dtype('int64'),
 'NUMBERO_OF_BANK_ACCOUNTS': dtype('int64'),
 'NUMBER_OF_SPECIAL_BANK_ACCOUNTS': dtype('int64'),
 'P_ASSETS_VALUE': dtype('float64'),
 'CARS_FLAG': dtype('int64'),
 'M_IN_THE_JOB': dtype('int64'),
 'C_PRODUCT': dtype('int64'),
 'AGE': dtype('int64'),
 'L_BALANCE': dtype('int64'),
 'MEAN_PAY_AMOUNT_MONTH': dtype('float64'),
 'MEAN_BILL_AMOUNT_MONTH': dtype('float64'),
 'MEAN_PAY_MONTH': dtype('float64'),
 'MEAN_RES_CITY': dtype('float64'),
 'MEAN_PROF_CITY': dtype('float64'),
 'EDU_LEVEL': dtype('float64'),
 'APP_SUB_TYPE=W': dtype('int64'),
 'APP_SUB_TYPE=Z': dtype('int64'),
 'APP_SUB_TYPE=C': dtype('int64')

# Feature selection

In [51]:
train_df = df_merged2[df_merged2['train_test'] == 'train'].copy()
test_df = df_merged2[df_merged2['train_test'] == 'test'].copy()

In [52]:
from sklearn.linear_model import Lasso
model = Lasso()

In [56]:
model.fit(train_df[bemeno_valtozok], train_df['TARGET'])
coeffs = pd.DataFrame({'att': bemeno_valtozok, 'coeffs':model.coef_}, index=bemeno_valtozok)

In [57]:
coeffs

,att,coeffs
DAY_OF_PAYMENT,DAY_OF_PAYMENT,0.000000e+00
M_STATUS,M_STATUS,-0.000000e+00
NUMBER_OF_DEPENDANTS,NUMBER_OF_DEPENDANTS,0.000000e+00
NAT,NAT,-0.000000e+00
MONTHS_IN_RES,MONTHS_IN_RES,-0.000000e+00
P_MONTHLY_INCOME,P_MONTHLY_INCOME,1.442031e-07
O_INCOMES,O_INCOMES,-6.419202e-07
VISA_FLAG,VISA_FLAG,0.000000e+00
MCARD_FLAG,MCARD_FLAG,-0.000000e+00
S_FLAG,S_FLAG,0.000000e+00


In [62]:
szurt_bemeno_valtozok = []
for att in bemeno_valtozok:
    if coeffs.loc[att, 'coeffs'] != 0:
        szurt_bemeno_valtozok.append(att)

In [63]:
szurt_bemeno_valtozok

['P_MONTHLY_INCOME',
 'O_INCOMES',
 'P_ASSETS_VALUE',
 'L_BALANCE',
 'MEAN_PAY_AMOUNT_MONTH',
 'MEAN_BILL_AMOUNT_MONTH']

# Anomália detekció

In [65]:
from sklearn.ensemble import IsolationForest

In [67]:
model = IsolationForest()

In [68]:
model.fit(train_df[bemeno_valtozok])
train_df['anomaly'] = model.predict(train_df[bemeno_valtozok])

In [71]:
train_df = train_df[train_df['anomaly'] == 1].copy()

# Modellezés

In [42]:
from sklearn.ensemble import RandomForestClassifier
model_rfc = RandomForestClassifier(random_state=42, n_estimators=300, max_depth=15)

In [43]:
from sklearn.ensemble import GradientBoostingClassifier
model_gbc = GradientBoostingClassifier(random_state=42, n_estimators=150, max_depth=10)

In [72]:
myxval([model_rfc, model_gbc], train_df, bemeno_valtozok)

0.7425796646088623

In [73]:
train_df

,ID,DAY_OF_PAYMENT,APP_SUB_TYPE,P_ADDRESS_TYPE,SEX,M_STATUS,NUMBER_OF_DEPENDANTS,BIRTH_STATE,BIRTH_CITY,NAT,RES_STATE,RES_CITY,RES_BOROUGH,HOME_PHONE_FLAG,RES_PHONE_AREA_CODE,RES_TYPE,MONTHS_IN_RES,EMAIL_FLAG,P_MONTHLY_INCOME,O_INCOMES,VISA_FLAG,MCARD_FLAG,S_FLAG,AME_FLAG,OCARDS_FLAG,NUMBERO_OF_BANK_ACCOUNTS,NUMBER_OF_SPECIAL_BANK_ACCOUNTS,P_ASSETS_VALUE,CARS_FLAG,C_NAME_GIVEN,PROF_STATE,PROF_CITY,PROF_BOROUGH,PROF_PHONE_GIVEN,PROF_PHONE_AREA_CODE,M_IN_THE_JOB,PROF_CODE,OCC_TYPE,MATE_PROF_CODE,EDU_LEVEL,CONF_CITIZEN_CARD_NUM,CONF_TAX_STATUS,C_PRODUCT,AGE,RES_ZIP,PROF_ZIP,L_BALANCE,PAY_MONTH_0,PAY_MONTH_2,PAY_MONTH_3,...,BILL_AMOUNT_MONTH_2,BILL_AMOUNT_MONTH_3,BILL_AMOUNT_MONTH_4,BILL_AMOUNT_MONTH_5,BILL_AMOUNT_MONTH_6,PAY_AMOUNT_MONTH_1,PAY_AMOUNT_MONTH_2,PAY_AMOUNT_MONTH_3,PAY_AMOUNT_MONTH_4,PAY_AMOUNT_MONTH_5,PAY_AMOUNT_MONTH_6,TARGET,train_test,MEAN_RES_CITY,MEAN_PROF_CITY,MEAN_BILL_AMOUNT_MONTH,MEAN_PAY_MONTH,MEAN_PAY_AMOUNT_MONTH,APP_SUB_TYPE=W,APP_SUB_TYPE=Z,APP_SUB_TYPE=C,P_ADDRESS_TYPE=A,P_ADDRESS_TYPE=B,SEX=female,SEX=male,SEX=nincs_adat,RES_TYPE=1.0,RES_TYPE=2.0,RES_TYPE=5.0,RES_TYPE=nincs_adat,RES_TYPE=0.0,C_NAME_GIVEN=no,C_NAME_GIVEN=yes,PROF_CODE=9.0,PROF_CODE=nincs_adat,PROF_CODE=11.0,PROF_CODE=0.0,PROF_CODE=2.0,OCC_TYPE=2.0,OCC_TYPE=1.0,OCC_TYPE=nincs_adat,OCC_TYPE=4.0,OCC_TYPE=5.0,MATE_PROF_CODE=nincs_adat,MATE_PROF_CODE=0.0,MATE_PROF_CODE=11.0,MATE_PROF_CODE=9.0,MATE_PROF_CODE=16.0,xval,anomaly
0,1,5,W,A,female,6,1,State0,City0,1,State0,City2115,Borough0,yes,105,1,15.0,1,245200.0,0.0,1,1,0,0,0,0,0,0.000000e+00,0,no,NaN,NaN,NaN,No,,0,9,4,nincs_adat,0.0,0,0,1,32,595,595,150000,-2,-2,-2,...,25988,3177,49174,11823,669,26034,3182,49174,11823,669,1534,1.0,train,0.405204,0.398107,15835.333333,-2.000000,15402.666667,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1
1,2,15,C,A,female,2,0,State1,City1,1,State1,City1,Borough1,yes,20,1,1.0,1,203200.0,0.0,0,0,0,0,0,0,0,0.000000e+00,0,yes,NaN,NaN,NaN,No,,0,11,4,11,0.0,0,0,1,34,230,230,180000,-2,-2,-2,...,9861,11021,16883,150,4661,9861,11021,16883,150,4661,873,1.0,train,0.371105,0.398107,7946.000000,-2.000000,7241.500000,0,0,1,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,2,1
2,3,20,W,A,female,2,0,State2,City3,1,State2,City3,NaN,no,,nincs_adat,0.0,1,133200.0,0.0,0,0,0,0,0,0,0,0.000000e+00,0,no,NaN,NaN,NaN,No,,0,nincs_adat,nincs_adat,nincs_adat,0.0,0,0,1,61,545,545,120000,0,0,0,...,101458,103840,105646,86127,67269,5100,5500,5170,3543,3000,3238,0.0,train,0.520000,0.398107,93721.500000,0.000000,4258.500000,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1
3,4,25,W,A,female,2,0,State7,City9,1,State7,City16,Borough10,yes,107,1,10.0,1,49200.0,0.0,0,1,0,0,0,0,0,0.000000e+00,0,yes,NaN,NaN,NaN,No,,0,9,4,nincs_adat,0.0,0,0,1,36,607,607,50000,0,0,2,...,48101,46993,47940,50903,49794,3855,0,1694,3713,2000,1900,1.0,train,0.455285,0.398107,48116.833333,0.666667,2193.666667,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1
4,5,15,W,A,female,1,1,State3,City10,1,State3,City10,Borough11,yes,32,1,6.0,1,106040.0,0.0,0,0,0,0,0,0,0,0.000000e+00,0,yes,State3,City10,Borough0,Yes,32,0,9,2,nincs_adat,0.0,0,0,2,29,384,384,100000,-2,-2,-2,...,1780,0,0,0,0,4780,0,0,0,0,0,0.0,train,0.350877,0.333333,361.666667,-2.000000,796.666667,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,24996,15,Z,A,male,1,0,State1,City1,1,State1,City7152,Borough3551,yes,22,1,15.0,0,166800.0,0.0,0,0,0,0,0,1,1,0.000000e+00,1,yes,State1,City7152,Borough1032,Yes,22,0,9,0,0,0.0,0,0,1,43,286,286,150000,0,0,0,...,125063,118843,137751,141330,146389,10076,10050,30000,